In [6]:
from pandas import read_csv
from matplotlib import pyplot
import seaborn as sn
import matplotlib.pyplot as plt

import numpy as np
from sklearn import svm

In [19]:
# MMD implementation
# https://github.com/jindongwang/transferlearning/blob/master/code/distance/mmd_numpy_sklearn.py

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Transfer learning project/UCR_TS_Archive_2015/')

In [4]:
dataset_list = ['50words/50words_TRAIN', 'Meat/Meat_TRAIN', 'HandOutlines/HandOutlines_TRAIN', 
                'InlineSkate/InlineSkate_TRAIN', 'DiatomSizeReduction/DiatomSizeReduction_TRAIN']

data = []
for data_name in dataset_list:
  data.append(read_csv(data_name, header=None))

In [7]:
np.shape(data[1])

(60, 449)

In [5]:
"""
The algorithm is as follows:

Mix the two datasets. Apply label that indicate each example's origin.
Train a classifier on these merged data.
Measure the classifier's error e on a held-out test set.
Set PAD = 2 (1 − 2e)
"""

"\nThe algorithm is as follows:\n\nMix the two datasets. Apply label that indicate each example's origin.\nTrain a classifier on these merged data.\nMeasure the classifier's error e on a held-out test set.\nSet PAD = 2 (1 − 2e)\n"

In [25]:
def proxy_a_distance(source_X, target_X, verbose=False):
    """
    Compute the Proxy-A-Distance of a source/target representation
    """
    nb_source = np.shape(source_X)[0]
    nb_target = np.shape(target_X)[0]

    if verbose:
        print('PAD on', (nb_source, nb_target), 'examples')

    C_list = np.logspace(-7, 5, 10)
    # C_list = [0.0001, 10]

    if np.shape(source_X)[1] < np.shape(target_X)[1]:
      half_source, half_target = int(nb_source/2), int(nb_target/2)
      train_X = np.vstack((source_X.iloc[0:half_source, :np.shape(source_X)[1]], target_X.iloc[0:half_target, :np.shape(source_X)[1]]))
      train_Y = np.hstack((np.zeros(half_source, dtype=int), np.ones(half_target, dtype=int)))

      test_X = np.vstack((source_X.iloc[half_source:, :np.shape(source_X)[1]], target_X.iloc[half_target:, :np.shape(source_X)[1]]))
      test_Y = np.hstack((np.zeros(nb_source - half_source, dtype=int), np.ones(nb_target - half_target, dtype=int)))

    else:
      half_source, half_target = int(nb_source/2), int(nb_target/2)
      train_X = np.vstack((source_X.iloc[0:half_source, :np.shape(target_X)[1]], target_X.iloc[0:half_target, :np.shape(target_X)[1]]))
      train_Y = np.hstack((np.zeros(half_source, dtype=int), np.ones(half_target, dtype=int)))  

      test_X = np.vstack((source_X.iloc[half_source:, :np.shape(target_X)[1]], target_X.iloc[half_target:, :np.shape(target_X)[1]]))
      test_Y = np.hstack((np.zeros(nb_source - half_source, dtype=int), np.ones(nb_target - half_target, dtype=int)))


    best_risk = 1.0
    for C in C_list:
        clf = svm.SVC(C=C, kernel='linear', verbose=False)
        clf.fit(train_X, train_Y)

        train_risk = np.mean(clf.predict(train_X) != train_Y)
        test_risk = np.mean(clf.predict(test_X) != test_Y)

        if verbose:
            print('[ PAD C = %f ] train risk: %f  test risk: %f' % (C, train_risk, test_risk))

        if test_risk > .5:
            test_risk = 1. - test_risk

        best_risk = min(best_risk, test_risk)

    return 2 * (1. - 2 * best_risk)

In [26]:
proxy_a_distance(data[1], data[2], verbose=True)

PAD on (60, 370) examples
[ PAD C = 0.000000 ] train risk: 0.139535  test risk: 0.139535
[ PAD C = 0.000002 ] train risk: 0.139535  test risk: 0.139535
[ PAD C = 0.000046 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.001000 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.021544 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.464159 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 10.000000 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 215.443469 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 4641.588834 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 100000.000000 ] train risk: 0.000000  test risk: 0.000000


2.0

In [27]:
proxy_a_distance(data[1], data[3], verbose=True)

PAD on (60, 100) examples
[ PAD C = 0.000000 ] train risk: 0.375000  test risk: 0.375000
[ PAD C = 0.000002 ] train risk: 0.375000  test risk: 0.375000
[ PAD C = 0.000046 ] train risk: 0.375000  test risk: 0.375000
[ PAD C = 0.001000 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.021544 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.464159 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 10.000000 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 215.443469 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 4641.588834 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 100000.000000 ] train risk: 0.000000  test risk: 0.000000


2.0

In [28]:
proxy_a_distance(data[4], data[3], verbose=True)

PAD on (16, 100) examples
[ PAD C = 0.000000 ] train risk: 0.137931  test risk: 0.137931
[ PAD C = 0.000002 ] train risk: 0.137931  test risk: 0.137931
[ PAD C = 0.000046 ] train risk: 0.137931  test risk: 0.137931
[ PAD C = 0.001000 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.021544 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 0.464159 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 10.000000 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 215.443469 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 4641.588834 ] train risk: 0.000000  test risk: 0.000000
[ PAD C = 100000.000000 ] train risk: 0.000000  test risk: 0.000000


2.0